# Distributed Data Parallelism with PyTorch

In this section, we're going to explore how to use PyTorch's distributed data parallelism capabilities in the [DDP](https://docs.pytorch.org/docs/stable/notes/ddp.html) library to train a model across multiple GPUs. This is particularly useful for speeding up training processes by and reducing the time it takes to train large models.

## Communication Pattern w/ DDP

The basic idea of DDP is to replicate the model on each GPU and then synchronize gradients across all GPUs during the backward pass. Each process computes gradients independently, and then they are averaged across all processes. This allows for efficient training without the need for a central parameter server.

![DDP Communication pattern - image from original FSDP paper by Facebook](images/ddp.png) 

### Wiring up the communications

To enable this, the first thing we have to do is to set up the process group so that each process handling a replica can communicate with each other.   This is going to be done exactly as with our simple communication scripts we ran with `torchrun` in the [Torchrun section](2_Torchrun_and_distributed.ipynb)

So as with (say) the [reductions.py example](code/reductions.py), we need to set up the process group with something like:

```python
import os
import torch.distributed as dist

global_rank = int(os.environ.get("RANK", 0))
local_rank = int(os.environ.get("LOCAL_RANK", 0))
world_size = int(os.environ.get("WORLD_SIZE", 1))

device = torch.device(f"cuda:{local_rank}")
dist.init_process_group(backend="nccl", world_size=world_size, rank=global_rank, device_id=device)
```

and then of course call `dist.destroy_process_group()` at the end of the script to clean up.

### Wrapping the model

For actually doing the gradient synchronization and implementing that in the training loop, we're in luck!
Implementing that manually would take some doing, but DistributedDataParallel offers a very simple high-level API to do this - you simply wrap your model with `torch.nn.parallel.DistributedDataParallel` and PyTorch handles the 
communication steps for you.

So for instance, in the first, simple example we're going to look at, the single GPU code is essentially:

```python
    model = torch.nn.Linear(20, 1)
    model = model.to(device)
```

Whereas in the DDP code, we would do:

```python
    from torch.nn.parallel import DistributedDataParallel as DDP

    model = torch.nn.Linear(20, 1)
    model = model.to(device)
    model = DDP(self.model, device_ids=[self.gpu_id])
```

That's the only real change we need to make to the model code to enable distributed data parallelism.

However, we still have more work to do!

## Data Loading for Data Parallelism

The whole point of data parallelism is to split the data across the different processes, so that each process can work on a different subset of the data in parallel. If we just use the same DataLoader we were using before, every replica will read the whole data set and we'll just end up by doing the exact same training run as before but $P$ times, in parallel.  Plus there'll be communication overhead for the gradient reduction!

To avoid this, we need to use a `DistributedSampler` to ensure that each process gets a different subset of the data. The `DistributedSampler` will automatically partition the dataset across the different processes and ensure that each process only sees its own subset of the data.

The `DistributedSampler` will also shuffle the data, but it does so in a way that ensures that each process gets a different subset of the data, so we don't need to worry about shuffling the data ourselves.  So if we had `shuffle=True` in our DataLoader before, we can just remove that instead rely on the `DistributedSampler`.

So what that looks like for the simple example we'll see next, is that we would change the DataLoader code from:

```python
    loader = DataLoader(dataset, batch_size=batch_size, pin_memory=True, shuffle=True)
```

to: 
```python
    from torch.utils.data.distributed import DistributedSampler

    sampler = DistributedSampler(dataset, num_replicas=world_size, rank=global_rank, shuffle=True)
    loader = DataLoader(dataset, batch_size=batch_size, pin_memory=True, sampler=sampler)
```

## Summary

So basically our steps for taking advantage of DDP are:


1. **Initialize the process group**: This sets up the communication backend and initializes the distributed environment.   Get the `RANK`, `LOCAL_RANK`, and `WORLD_SIZE` from the environment variables, which are set by `torchrun` when launching the script, and call `dist.init_process_group()` with the appropriate parameters.
2. **Grab the right device**: Use the `LOCAL_RANK` to set the device for each process, so that each process uses a different GPU.
3. **Wrap the model**: Use `torch.nn.parallel.DistributedDataParallel` to wrap your model, which will handle the gradient synchronization.
4. **Data loading**: Use `torch.utils.data.distributed.DistributedSampler` to ensure that each process gets a unique subset of the data.
5. **Training loop**: Very little needs to be done here
6. **Synchronize only where necessary, and Gate I/O**: Where absolutely necessary use `dist.barrier()` - but be careful, it's a very expensive operation - and make sure you don't have multiple processes writing to the screen or to a file unless necessary, so you'll probably have `if global_rank == 0` or `if local_rank == 0` checks.
5. **Clean up**: Call `dist.destroy_process_group()` at the end of your script to clean up the process group.

## Simple example - from [PyTorch documentation](https://pytorch.org/tutorials/intermediate/ddp_tutorial.html)

Given what we've discused, let's take a look at the simple example from the Pytorch documentation.  Compare the single-GPU version with the DDP version.  

* [ddp-example-singlegpu.py](code/ddp-example-singlegpu.py)
* [ddp-example-multigpu.py](code/ddp-example-multigpu.py)

## EuroSAT example

Now we're going to take a look at a still simple but slightly more realistic example - we're going to take the single-GPU EuroSAT classifier traininer we looked at [in a previous notebook](1_EuroSAT_single_gpu.ipynb), and train it with DDP across a varying number of 

You can run the [single GPU implementation](code/eurosat_singlegpu.py) 
below, to see that it performs as with the interactive version 
we played with before.

In [ ]:
!python3 ./code/eurosat_singlegpu.py

We also have a marked-up version with some comments walking you through the process of adding DDP to the training of this model; you can look at it [here](code/eurosat_pipeline_step0.py).  You can run it to see that it gives the same results (all that's changed are some comments).  Make a note of the elapsed time, and training rate (images per second).

In [ ]:
!python ./code/eurosat_pipeline_step0.py

Lets save a copy of this before we start...  you can always copy it back if something goes wrong.

In [ ]:
!cp ./code/eurosat_pipeline_step0.py ./code/eurosat_pipeline_orig.py

### Step 1 - Adding communications infrastructure.

Edit [the code](code/eurosat_pipeline_step0.py) to take care of all of the 'TODO Step 1' steps.  Basically you're setting up the communications infastructure, assigning the appropriate GPUs, and only having one rank print out status updates or write checkpoints:  that means


1. **Initialize the process group**: This sets up the communication backend and initializes the distributed environment.   Get the `RANK`, `LOCAL_RANK`, and `WORLD_SIZE` from the environment variables, which are set by `torchrun` when launching the script, and call `dist.init_process_group()` with the appropriate parameters.
2. **Grab the right device**: Use the `LOCAL_RANK` to set the device for each process, so that each process uses a different GPU.
3. **Synchronize only where necessary, and Gate I/O**: Where absolutely necessary use `dist.barrier()` - but be careful, it's a very expensive operation - and make sure you don't have multiple processes writing to the screen or to a file unless necessary, so you'll probably have `if global_rank == 0` or `if local_rank == 0` checks.
4. **Clean up**: Call `dist.destroy_process_group()` at the end of your script to clean up the process group.

When you're done, you will be able to safely run this with torchrun!  Run the command below and make sure you're looking at GPU utilisation, elapsed time, and training rates:

In [ ]:
!./code/run_w_torchrun.sh 2 ./code/eurosat_pipeline_step0.py

You can try other numbers of processes, too.   If you're having trouble, you can [look at the ansewr here](code/answers/eurosat_pipeline_step1.py)

Hopefully, it worked, and processes weren't stepping over each other's output.  Great!  But you won't have noticed any speedup - right now we have $P$ processes, all running exactly the same training job over the same data!  Let's do better.

### Step 2 - Model Wrapping and Data sampling

We've done some of the 'distributed' part, but none of the 'data parallel' part yet.  So now it's time to work on the Step 2 TODOs.

We're going to:

1. **Wrap the model**: Import DistributedDataParallel from `torch.nn.parallel` and use it to wrap our model.
2. **Create data samplers**: We'll need [DistributedSampler](https://docs.pytorch.org/docs/stable/data.html#torch.utils.data.distributed.DistributedSampler) from `torch.utilities.data.distributed`, and we'll create train and test data samplers.  Note we have to pass our rank and the world size (number of replicas), because the sampler needs to divide the data evenly between all the participating processes.
3. **Load with sample**: We need to add `sampler=train_smapler` (or test_sampler) to the dataloader call, so that the data is sampled appropriately.  We can use that to replace `shuffle=True`, if we're using it, as the data will be shuffled by the sampler, not the loader.   
4. **Update the sampler epoch**: Finally, at the start of each epoch, we'll have to set thet epoch for the train_sampler (`train_sampler.set_epoch(epoch)`) so that the data gets re-shuffled each epoch.   We don't really need that for the test sampler.

Give it a try!  First, save the work you've already got:

In [ ]:
!cp code/eurosat_pipeline_step0.py ./code/eurosat_pipeline_step1.py

And fill in the stage 2 TODOs [in your working file](code/eurosat_pipeline_step0.py).  Test it out with:

In [ ]:
!./code/run_w_torchrun.sh 2 ./code/eurosat_pipeline_step0.py

and see if you get some speedups!  If you're stuck, you can always look at [the step2 answers](code/answers/eurosat_pipeline_step2.py) or [the final solution](code/answers/eurosat_ddp_finished.py).  You can also run the completed version here:

In [ ]:
!./code/run_w_torchrun.sh 2 ./code/answers/eurosat_pipeline_finished.py

## Results


Congratulations!  You should now have a fully-working DDP implementation of the EuroSAT trainer.   Run with a number of processes from 1..4.  Do you see a linear speedup (e.g. images per second go up proportionally with number of replicas)?  Why or why not?

## Optional Next Steps

There's lots of places you could go with this.

* **Improve scaling**: right now the scaling is good but not great; the communications overhead starts being comparable to the computation you're saving with such a small model.   Try making the model a little bigger - add another convolutional block or two.   That should slow things down, but does it improve the relative speedup?  Why or why not?
* **Optional BatchNorm**: there are some operations like batch-norm which normalize results by batch.  But that's nontrivial here; a single "batch" is really $P$ times as a big as it was before, because $P$ replicas are each looking at their own microbatch before syncing.   If you wrap the model with `nn.SyncBatchNorm.convert_sync_batchnorm(net)`, there will be some synchronization to normalize results over the entire minibatch.   Does this improve accuracy?  How does it affect speed?
* **Batch Size** It's always worth playing with batch size.  Try cranking it up.  You might find that at otherwise-reasonable batch sizes, especially with 4 replicas, things start blowing up.  As above, a minibatch is now the sum of batch sizes of all the replicas.  Sometimes data parallel approaches benefit from a more sophisticated optimzier than SGD.  Try some other optimizers and see if you can increase the batch size further.
